<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/yolov5_gradCAM_corneAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall deep_utils -y
!pip install -U git+https://github.com/pooya-mohammadi/deep_utils.git
!pip install torch
!pip install torchvision
!pip install -U opencv-python


print("[INFO] To use new installed version of opencv, the session should be restarted!!!!")

In [ ]:
!git clone https://github.com/pooya-mohammadi/yolov5-gradcam

In [ ]:
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/3.jpg"

In [ ]:
import os
os.chdir('/content/yolov5-gradcam')

In [ ]:
!python main.py --model-path $model_path --img-path $img_path --output-dir out

In [ ]:
from PIL import Image
Image.open("out/eagle-res.jpg")